<h1><center><b>Moneypoly</b></center></h1>

# **Contexto**

Este es el producto final del prototipo "Moneypoly" y tercera parte del desarrollo

Primera parte:
https://www.kaggle.com/code/maximilianoalarcon/hackathon-bbva-2022-eda-feature-engineering

Segunda parte:
https://www.kaggle.com/code/maximilianoalarcon/hackathon-bbva-2022-model-development/notebook

En este notebook usted podra ingresar un dataset de bienes raices para calcular sus valorizaciones monetarias.

Primero debe copiar este notebook presionando el boton negro "Edit" ubicado en la parte superior derecha

Luego podra interactuar con este notebook y ejecutar secuencialmente los bloques de codigo, en un bloque particular podra ingresar un dataset desde su maquina local

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

df_train = pd.read_csv("../input/hackathonbbva2022datadriven/train_v1.csv")
df_test = pd.read_excel("../input/hackathonbbva2022datadriven/dataset_test_pruebas.xlsx")

def number_betweeen_in_range(num,since,until):
    return (num >= since) and (num <= until)

def extraer_valores_principales(column,rango=(0.7,1.3)):
    lista_valores = column.values.reshape(1, -1).ravel().tolist()
    lista_extraida = []
    if len(lista_valores) >= 3:    
        for i in range(len(lista_valores)-2):
            i_1 = i
            i_2 = i+2
            cociente = lista_valores[i_1:i_2][0]/ lista_valores[i_1:i_2][1]
            if  number_betweeen_in_range(cociente,rango[0],rango[1]):
                lista_extraida.append(lista_valores[i_1:i_2][0])
            else:
                lista_extraida.append(lista_valores[i_1:i_2][0])
                break;
    else:
        lista_extraida.append(lista_valores[0])

    if len(lista_extraida) < 1:
        lista_extraida.append(lista_valores[0])
    return lista_extraida

def estimacion_por_costo(group,ejemplar):
    cols_costo = ['Área Construcción','Área Terreno',"Edad",'Número de estacionamiento',"Depósitos"]
    target = ["Valor comercial (USD)"]
    dataset_costo = group[cols_costo+target].copy()
    for c in cols_costo:
        dataset_costo[c] = abs(dataset_costo[c]-ejemplar[c])
    dataset_costo.sort_values(cols_costo,inplace=True)
    estimacion = 0
    suma_diff = 0
    for c in cols_costo:
        suma_diff +=  dataset_costo[c].iloc[0]
    if suma_diff == 0:
        estimacion = dataset_costo.iloc[0,len(cols_costo)+len(target)-1]
    else:
        estimacion = np.mean(extraer_valores_principales(dataset_costo["Valor comercial (USD)"]))
    return estimacion

def estimacion_por_comparacion(group,ejemplar): 
    departamento = ejemplar["Departamento"]
    provincia = ejemplar["Provincia"]
    distrito = ejemplar["Distrito"]
    mask_1 = group["Departamento"] == departamento
    mask_2 = group["Provincia"] == provincia
    mask_3 = group["Distrito"] == distrito
    if sum(mask_1*mask_2*mask_3) > 0:
        group = group[mask_1*mask_2*mask_3]
    elif sum(mask_1*mask_2) > 0:
        group = group[mask_1*mask_2]
    elif sum(mask_1) > 0:
        group = group[mask_1]
    
    cols_costo = ['Latitud (Decimal)', 'Longitud (Decimal)', 'Edad', 'Área Terreno', 'Área Construcción']
    target = ["Valor comercial (USD)"]
    dataset_costo = group[cols_costo+target].copy()
    for c in cols_costo:
        dataset_costo[c] = abs(dataset_costo[c]-ejemplar[c])
    dataset_costo.sort_values(cols_costo,inplace=True)
    estimacion = 0
    suma_diff = 0
    for c in cols_costo:
        suma_diff +=  dataset_costo[c].iloc[0]
    if suma_diff == 0:
        estimacion = dataset_costo.iloc[0,len(cols_costo)+len(target)-1]
    else:
        estimacion = np.mean(extraer_valores_principales(dataset_costo["Valor comercial (USD)"]))
    return estimacion

def estimacion_por_renta_capitalizacion(group,ejemplar):
    tipo_via = ejemplar["Tipo de vía"]
    mask_1 = group["Tipo de vía"] == tipo_via
    if sum(mask_1) > 0:
        group = group[mask_1]
    cols_costo = ["Edad",'Área Terreno','Área Construcción','Número de estacionamiento',"Depósitos"]
    target = ["Valor comercial (USD)"]
    dataset_costo = group[cols_costo+target].copy()
    for c in cols_costo:
        dataset_costo[c] = abs(dataset_costo[c]-ejemplar[c])
    dataset_costo.sort_values(cols_costo,inplace=True)
    estimacion = 0
    suma_diff = 0
    for c in cols_costo:
        suma_diff +=  dataset_costo[c].iloc[0]
    if suma_diff == 0:
        estimacion = dataset_costo.iloc[0,len(cols_costo)+len(target)-1]
    else:
        estimacion = np.mean(extraer_valores_principales(dataset_costo["Valor comercial (USD)"]))
    return estimacion

types = {'Categoría del bien': 'object',
 'Depósitos': 'float64',
 'Edad': 'float64',
 'Estado de conservación': 'object',
 'Latitud (Decimal)': 'float64',
 'Longitud (Decimal)': 'float64',
 'Método Representado': 'object',
 'Número de estacionamiento': 'float64',
 'Área Construcción': 'float64',
 'Área Terreno': 'float64',
 'Tipo de vía': 'object',
 'Departamento': 'object',
 'Provincia': 'object',
 'Distrito': 'object'}


map_tipo_via = {
    1:"CALLE",
    2:"JIRON",
    3:"AVENIDA",
    4:"PASAJE",
    5:"CARRETERA",
    6:"MANZANA",
    7:"LOTE",
    8:"PARCELA",
    9:"FUNDO",
    10:"SIN_VIA",
    "1.0":"CALLE",
    "2.0":"JIRON",
    "3.0":"AVENIDA",
    "4.0":"PASAJE",
    "5.0":"CARRETERA",
    "6.0":"MANZANA",
    "7.0":"LOTE",
    "8.0":"PARCELA",
    "9.0":"FUNDO",
    "10.0":"SIN_VIA"
}

cant_decimales = 3

df_centros = pd.read_csv("../input/hackathonbbva2022datadriven/centros_lat_lon.csv")
departamentos_conocidos = list(df_centros["departamento"].unique())
df_centros.set_index(["departamento"],inplace=True)

categorias_conocidas = list(df_train["Categoría del bien"].unique())

means_areas = {}
for index,group in df_train.groupby(["Categoría del bien"]):
    means_areas[index] = {}
    means_areas[index]["mean_area_terreno"] = round(group["Área Terreno"].mean(),cant_decimales)
    means_areas[index]["mean_area_construccion"] = round(group["Área Construcción"].mean(),cant_decimales)

def validar_categoria(categoria):
    if categoria in categorias_conocidas:
        return categoria
    else:
        return "Departamento"
    
def validar_departamento(departamento):
    if departamento in departamentos_conocidos:
        return departamento
    else:
        return "Lima"    
    
def busqueda_latitud(x):
    return df_centros.loc[x["Departamento"]]["lat_median"]
    
def busqueda_longitud(x):
    return df_centros.loc[x["Departamento"]]["lon_median"]

def busqueda_area_terreno(categoria,area_construccion):
    mask1 = df_train["Categoría del bien"] == categoria
    mask2 = df_train["Área Terreno"].notnull()
    mask = mask1*mask2
    df_train_filtrado = df_train[mask].copy()
    df_train_filtrado["diff"] = abs(df_train_filtrado["Área Construcción"] - area_construccion)
    mask = df_train_filtrado["diff"] == df_train_filtrado["diff"].min()
    df_train_filtrado = df_train_filtrado[mask].copy()
    return df_train_filtrado["Área Terreno"].mean()

def busqueda_area_construccion(categoria,area_terreno):
    mask1 = df_train["Categoría del bien"] == categoria
    mask2 = df_train["Área Construcción"].notnull()
    mask = mask1*mask2
    df_train_filtrado = df_train[mask].copy()
    df_train_filtrado["diff"] = abs(df_train_filtrado["Área Terreno"] - area_terreno)
    mask = df_train_filtrado["diff"] == df_train_filtrado["diff"].min()
    df_train_filtrado = df_train_filtrado[mask].copy()
    return df_train_filtrado["Área Construcción"].mean()

def transformacion(df):
    df_copy = df.copy()
    atributos = ['Categoría del bien','Depósitos','Edad','Estado de conservación','Latitud (Decimal)','Longitud (Decimal)',
                 'Método Representado','Número de estacionamiento','Área Construcción','Área Terreno',"Tipo de vía","Departamento",
                 "Provincia","Distrito"]
    if "ID" in df.columns:
        atributos = ["ID"]+atributos
    df_copy = df_copy[atributos].copy()
    
    cols_categoricas = ["Categoría del bien","Estado de conservación","Método Representado","Tipo de vía","Departamento","Provincia","Distrito"]
    for c in cols_categoricas:
        if str(df_copy[c].dtypes) != types[c]:
            df_copy[c] = df_copy[c].astype("object")
            
    cols_numericas = ["Depósitos","Edad",'Número de estacionamiento','Área Construcción','Área Terreno']
    for c in cols_numericas:
        df_copy[c] = df_copy[c].astype(str)
        df_copy[c] = df_copy[c].str.replace(',','')
        df_copy[c] = pd.to_numeric(df_test[c],errors="coerce")
    
    cols_numericas_esp = ['Latitud (Decimal)','Longitud (Decimal)']
    for c in cols_numericas_esp:
        if df_copy[c].dtypes != "float64":
            df_copy[c] = df_copy[c].astype(str)
            df_copy[c] = df_copy[c].str.replace(',','')
            df_copy[c] = pd.to_numeric(df_test[c],errors="coerce")        
    
    df_copy["Tipo de vía"] = df_copy["Tipo de vía"].map(map_tipo_via)
    
    df_copy["Tipo de vía"] = df_copy["Tipo de vía"].fillna("CALLE")
    df_copy["Número de estacionamiento"] = df_copy["Número de estacionamiento"].fillna(0)
    df_copy["Depósitos"] = df_copy["Depósitos"].fillna(0)
    df_copy["Método Representado"] = df_copy["Método Representado"].fillna("Comparación de mercado (directo)")
    df_copy["Categoría del bien"] = df_copy["Categoría del bien"].fillna("Departamento")
    df_copy["Edad"] = df_copy["Edad"].fillna(0)
    df_copy["Estado de conservación"] = df_copy["Estado de conservación"].fillna("Bueno")
    
    df_copy["Departamento"] = df_copy["Departamento"].fillna("Lima")
    df_copy["Provincia"] = df_copy["Provincia"].fillna("Lima")
    df_copy["Distrito"] = df_copy["Distrito"].fillna("Santiago de Surco")
    
    
    #Imputacion de latitud y longitud
    lista_latitudes = []
    lista_longitudes = []
    for index,row in df_copy.iterrows():
        if pd.isnull(row["Latitud (Decimal)"]):     
            row["Departamento"] = validar_departamento(row["Departamento"])
            lista_latitudes.append(busqueda_latitud(row))
        else:
            lista_latitudes.append(row["Latitud (Decimal)"])
        if pd.isnull(row["Longitud (Decimal)"]):     
            row["Departamento"] = validar_departamento(row["Departamento"])
            lista_longitudes.append(busqueda_longitud(row))
        else:
            lista_longitudes.append(row["Longitud (Decimal)"])
    df_copy["Latitud (Decimal)"] = pd.Series(lista_latitudes)
    df_copy["Longitud (Decimal)"] = pd.Series(lista_longitudes)
    
    
    #Imputacion de areas construccion
    mask1 = df_copy["Área Construcción"].isnull()
    mask2 = df_copy["Categoría del bien"].notnull()
    mask3 = df_copy["Área Terreno"].notnull()
    mask = mask1*mask2*mask3
    areas_construcciones = []
    if sum(mask) > 0:
        df_copy["mask"] = mask
        for index,row in df_copy.iterrows():
            if row["mask"]:
                busqueda = busqueda_area_construccion(validar_categoria(row["Categoría del bien"]),row["Área Terreno"])
                areas_construcciones.append(round(busqueda,cant_decimales))
            else:
                areas_construcciones.append(round(row["Área Construcción"],cant_decimales))
        df_copy["Área Construcción"] = areas_construcciones
        df_copy.drop(["mask"],inplace=True,axis=1)
    
    
    #Imputacion de areas terrenos
    mask1 = df_copy["Área Terreno"].isnull()
    mask2 = df_copy["Categoría del bien"].notnull()
    mask3 = df_copy["Área Construcción"].notnull()
    mask = mask1*mask2*mask3
    areas_terrenos = []
    if sum(mask) > 0:
        df_copy["mask"] = mask
        for index,row in df_copy.iterrows():
            if row["mask"]:
                busqueda = busqueda_area_terreno(validar_categoria(row["Categoría del bien"]),row["Área Construcción"])
                areas_terrenos.append(round(busqueda,cant_decimales))
            else:
                areas_terrenos.append(round(row["Área Terreno"],cant_decimales))
        df_copy["Área Terreno"] = areas_terrenos
        df_copy.drop(["mask"],inplace=True,axis=1)
    
    return df_copy

    mask1 = df_copy["Área Terreno"].isnull()
    mask2 = df_copy["Área Construcción"].isnull()
    mask = mask1*mask2
    areas_terrenos = []
    areas_construcciones = []
    if sum(mask) > 0:
        df_copy["mask"] = mask
        for index,row in df_copy.iterrows():
            if row["mask"]:
                row["Categoría del bien"] = validar_categoria(row["Categoría del bien"])
                areas_terrenos.append(means_areas[row["Categoría del bien"]]["mean_area_terreno"])
                areas_construcciones.append(means_areas[row["Categoría del bien"]]["mean_area_construccion"])
        df_copy["Área Terreno"] = areas_terrenos
        df_copy["Área Construcción"] = areas_construcciones
        
        
from tqdm import tqdm

def model(df):
    estimaciones = []
    for index,row in tqdm(df.iterrows(), total=df.shape[0]):
        metodo = row["Método Representado"]
        estado = row["Estado de conservación"]
        categoria = row["Categoría del bien"]
        mask_1 = df_train["Método Representado"] == metodo
        mask_2 = df_train["Estado de conservación"] == estado
        mask_3 = df_train["Categoría del bien"] == categoria
        group = None
        
        masks = []
        list_masks = {}
        masks.append({
            "indice":0,"suma":sum(mask_1*mask_2*mask_3)
        })
        list_masks[0] = mask_1*mask_2*mask_3
        masks.append({
            "indice":1,"suma":sum(mask_1*mask_2)
        })
        list_masks[1] = mask_1*mask_2
        masks.append({
            "indice":2,"suma":sum(mask_1*mask_3)
        })
        list_masks[2] = mask_1*mask_3
        masks.append({
            "indice":3,"suma":sum(mask_2*mask_3)
        })
        list_masks[3] = mask_2*mask_3
        masks.append({
            "indice":4,"suma":sum(mask_1)
        })
        list_masks[4] = mask_1
        masks.append({
            "indice":5,"suma":sum(mask_2)
        })
        list_masks[5] = mask_2
        masks.append({
            "indice":6,"suma":sum(mask_3)
        })
        list_masks[6] = mask_3
        masks = pd.DataFrame(masks)
        masks = masks[masks["suma"] > 0]
        masks.sort_values(["suma"],inplace=True)
        if masks.shape[0] > 0:
            group = df_train[list_masks[masks["indice"][0]]]  
        else:
            group = df_train.copy()
        
        if metodo == "Costos o reposición (directo)":
            estimaciones.append(int(estimacion_por_costo(group,row)))    
        elif metodo == "Comparación de mercado (directo)":
            estimaciones.append(int(estimacion_por_comparacion(group,row))) 
        elif metodo == "Renta o capitalización (indirecto)":
            estimaciones.append(int(estimacion_por_renta_capitalizacion(group,row)))
        else:
            estimaciones.append(int(estimacion_por_comparacion(group,row)))
    return estimaciones

from IPython.display import display, FileLinks
import base64
import pandas as pd
from IPython.display import HTML

def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [2]:
from ipywidgets import FileUpload
upload = FileUpload(
    accept='.xlsx',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
print("Cargue su dataset aqui por favor")
print("="*64)
print("")
display(upload)

Cargue su dataset aqui por favor



FileUpload(value={}, accept='.xlsx', description='Upload')

In [3]:
if len(upload.data) > 0:
    with open("test.xlsx", "w+b") as i:
        i.write(upload.data[0]) 
    df_test = pd.read_excel("test.xlsx")
    print("Dataset cargado")
    print("="*64)
    print("")
    print("")
    display(df_test.head())
    print("Aplicando transformacion de datos......")
    print("")
    print("")
    df_test = transformacion(df_test)
    print("Calculando las proyecciones......")
    print("="*64)
    df_test["Valor comercial (USD)"] = model(df_test)
    print("Listo!!")
    display(df_test.head())
    print("Puede descargar las proyecciones con el siguiente link")
    print("")
    display(create_download_link(df_test, filename="projections.csv"))
    print("")
else:
    print("Por favor, cargue un dataset con la siguiente estructura")
    print("")
    display(df_test.head())

Por favor, cargue un dataset con la siguiente estructura



,ID,Fecha entrega del Informe,Tipo de vía,Piso,Departamento,Provincia,Distrito,Número de estacionamiento,Depósitos,Latitud (Decimal),...,Categoría del bien,Posición,Número de frentes,Edad,Elevador,Estado de conservación,Método Representado,Área Terreno,Área Construcción,Valor comercial (USD)
0,1.0,2021-05-21,1.0,NaN,Lima,Lima,Surquillo,0.0,1.0,-12.105644,...,Departamento,NaN,NaN,1.0,NaN,Bueno,Comparación de mercado (directo),11.0,64.92,NaN
1,2.0,2017-11-26,2.0,NaN,Lima,Lima,Santiago de Surco,1.0,1.0,-12.138068,...,Departamento,NaN,NaN,1.0,NaN,Muy bueno,Comparación de mercado (directo),50.32,134.20,NaN
2,3.0,2019-03-01,3.0,NaN,Apurímac,Chincheros,Chincheros,1.0,1.0,NaN,...,Oficina,NaN,NaN,1.0,NaN,Bueno,Comparación de mercado (directo),2022-08-05 00:00:00,111.11,NaN
3,4.0,2019-11-22,2.0,NaN,Ucayali,Coronel Portillo,Calleria,NaN,NaN,-8.381362,...,Terreno Urbano,NaN,NaN,NaN,NaN,NaN,Costos o reposición (directo),31.0,0.00,NaN
4,5.0,2020-10-16,1.0,NaN,Lima,Lima,Magdalena del Mar,1.0,2.0,-12.091392,...,NaN,NaN,NaN,2.0,NaN,En proyecto,Comparación de mercado (directo),3.22,111.11,NaN
